In [1]:
from collections import Counter as counter
import matplotlib.pyplot as plt
import csv
import numpy as np
import os
import scipy
import matplotlib.path as mplPath
import random
from matplotlib.ticker import NullFormatter, MaxNLocator
from numpy import linspace
import urllib2
import json
import unidecode
import re
import math
import sys
from six.moves import cPickle as pickle
import glob

FUNCTIONS

In [2]:
def pointInsidePolygon(point, polygon):
    X = [x[0] for x in polygon]
    Y = [x[1] for x in polygon]
    if point[0]>=min(X) and point[0]<=max(X) and \
       point[1]>=min(Y) and point[1]<=max(Y):
        polygon = mplPath.Path(np.array(polygon))
        return polygon.contains_point(point)
    else:
        return 0

# Used to determine if a particular timestamp falls within a set of timeslots
def inAllSlots(timeStamp, allSlots):
    for slot in allSlots:
        if (timeStamp-slot[0])*(timeStamp-slot[1])<=0:
            return 1

# Function to convert a list into a compound list by gouping every n elements
def chunks(l, n):
    n = max(1, n)
    return [l[i:i + n] for i in range(0, len(l), n)]

def allocateHours(buildingCategory):
    if buildingCategory=='res':
        return [[[0.0,24.0]]]*7
    elif buildingCategory=='wo':
        return [[[7.0,21.0]]]*7
    elif buildingCategory=='i':
        return [[[7.0, 17.0]]]*5+[[[0.0, 0.0]]]*2
    elif buildingCategory=='None':
        return [[[0.0, 0.0]]]*7
    else:
        print 'Unidentified building category'
        print buildingCategory

CONSTANTS

In [3]:
tracts = glob.glob('../PickledData/TractStays/*.pickle')
tracts = [t.split('Stays/')[1].strip('.pickle') for t in tracts]

In [4]:
RESIDENTIAL_PCA = 40 # square meters
UNKNOWN_LOCATIONS_PCA = 10

tractBuildings = {}
for t in tracts:
    with open('../PickledData/TractBuildings/'+t+'.pickle', 'rb') as f:
        tractBuildings[t] = pickle.load(f)

tractPois = {}
for t in tracts:
    with open('../PickledData/TractPOIs/'+t+'.pickle', 'rb') as f:
        tractPois[t] = pickle.load(f)


The PCAs are only determined directly from the POIs where there was no information regarding use per floor type.
When this inofrmation was available, PCAs were used for verification and to update the uses list if required.
This required manual filtering

In [5]:
incompletePois = []
completePois = []
# Iterate over POIs, find building for each
total = 0
successfulMatches = 0
for tractIndex in range(len(tracts)):
    tract = tracts[tractIndex]
    pois = tractPois[tract]
    buildings = tractBuildings[tract]
    for poiIndex in range(len(pois)):
        total+=1
        poiId = pois.keys()[poiIndex]
        try:
            poiLocation = pois[poiId]['location']
            for buildingIndex in range(len(buildings)):
                buildingId = buildings.keys()[buildingIndex]
                buildingOutline = buildings[buildingId]['geometry']
                if pointInsidePolygon(poiLocation, buildingOutline):
                    successfulMatches += 1
                    completePois.append( [tract, poiId] )
                    break
        except:
            incompletePois.append( [tract, poiId] )
print 'There are', total, 'POIs'
print 'There are', successfulMatches, 'matched POIs'

There are 0 POIs
There are 0 matched POIs


In [6]:
#print incompletePois

In [7]:
# incompletePois
# del tractPois[incompletePois[0][0]][incompletePois[0][1]]

assign POIs to buildings

In [8]:
allPois = {}
for entry in completePois:
    allPois[entry[1]] = tractPois[entry[0]][entry[1]]

for tract in tracts:
    for building in tractBuildings[tract]:
        tractBuildings[tract][building]['pois'] = []

# Iterate over POIs, find building for each
for tractIndex in range(len(tracts)):
    tract = tracts[tractIndex]
    pois = tractPois[tract]
    buildings = tractBuildings[tract]
    for poiIndex in range(len(pois)):
        poiId = pois.keys()[poiIndex]
        try:
            poiLocation = pois[poiId]['location']
            for buildingIndex in range(len(buildings)):
                buildingId = buildings.keys()[buildingIndex]
                buildingOutline = buildings[buildingId]['geometry']
                if pointInsidePolygon(poiLocation, buildingOutline):
                    tractBuildings[tract][buildingId]['pois'].append(poiId)
        except:
            print 'Found incomplete an POI'

Save the POIs in the model area for future use

In [9]:
# # now save a csv file with ids, building info and location
# objectIds = []
# with open('../Data/SDL_Buildings/BBXX_USEAREA_POL.txt', 'r') as f:
#     f.readline()
#     for line in f:
#         line = line.replace("\"", "")
#         line = line.replace("\r\n", "")
#         line = line.split(',')
#         objectIds.append(line[0])
        
# allBuildings = {}
# for tract in tracts:
#     for bId in tractBuildings[tract].keys():
#         allBuildings[bId] = tractBuildings[tract][bId]

# filename = 'BB_building_pois.csv'
# with open('../Useful_Results/'+filename, 'wb') as f:
#     spamwriter = csv.writer(f, delimiter=',')
#     RESULT = []
#     RESULT.append('objectId')
#     RESULT.append('new Id')
#     RESULT.append('Use')
#     RESULT.append('Area (sq m)')
#     #RESULT.append('building outline')
#     RESULT.append('Poi Info')
#     spamwriter.writerow(RESULT) 
#     for oId in objectIds:
#         buildings = [x for x in allBuildings.keys() if x.split('_')[0]==oId]
#         for eachBuilding in buildings:
#             use = allBuildings[eachBuilding]['use']
#             outline = allBuildings[eachBuilding]['geometry']
#             area = allBuildings[eachBuilding]['useArea']
#             RESULT = []
#             RESULT.append(eachBuilding.split('_')[0])
#             RESULT.append(eachBuilding)
#             RESULT.append(use)
#             RESULT.append(area)
#             #RESULT.append(outline)
#             if allBuildings[eachBuilding]['pois']:
#                 poiList = []
#                 for poi in allBuildings[eachBuilding]['pois']:
#                     poiList.append('name: '+allPois[poi]['name'])
#                     poiTypes = [x for x in allPois[poi]['types'] if x not in ['point_of_interest', 'establishment']]              
#                     strTest = 'types: '+' '.join(poiTypes) 
#                     poiList.append(strTest)
#                     poiList.append('location: '+str(allPois[poi]['location'][0])+' '+str(allPois[poi]['location'][0]))
#                     if 'opening_hours' in allPois[poi].keys():
#                         #if 'Monday' in allPois[poi]['opening_hours'].keys():
#                         if allPois[poi]['opening_hours']['Monday']:
#                             #print allPois[poi]['opening_hours']['Monday']
#                             poiList.append('Mon opening:'+str(allPois[poi]['opening_hours']['Monday'][0][0])+'_to_'+ \
#                                            str(allPois[poi]['opening_hours']['Monday'][0][1]))
#                 RESULT.append(poiList)
#             spamwriter.writerow(RESULT) 

assign opening hours based on building type

In [10]:
usesSet = []
allObjectIDs = []
for tractIndex in range(len(tracts)):
    tract = tracts[tractIndex]
    buildings = tractBuildings[tract]
    # print len(buildings)
    for buildingID in buildings.keys():
        if buildings[buildingID]['use'] not in usesSet:
            usesSet.append(buildings[buildingID]['use'])
        oId = buildingID.split('_')[0]
        if oId not in allObjectIDs:
            allObjectIDs.append( oId )

In [11]:
# get the buildings where we have details and objectId and buildingId
areas = []
objectIDs = []
SDLIDs = []
with open('../Data/SDL_Buildings/BBXX_USEAREA_POL.txt', 'r') as f:
    f.readline()
    for line in f:
        line = line.replace("\"", "")
        line = line.replace("\r\n", "")
        line = line.split(',')
        # line[5] is polygon area and line[6] is tot floor area
        b = []
        for a in line[6:]:
            b.append(float(a))
        areas.append(b)
        oId = line[0]
        if oId in allObjectIDs:
            objectIDs.append(line[0])
            SDLIDs.append(line[1])

In [12]:
def allocateHours(buildingCategory):
    if buildingCategory=='OFFICE':
        return [[[6.0,21.0]]]*7
    elif buildingCategory=='RESIDENT':
        return [[[0.0,24.0]]]*7
    elif buildingCategory=='RESTRNT':
        return [[[5.0,24.0]]]*7
    elif buildingCategory=='HOTEL':
        return [[[0.0,24.0]]]*7
    elif buildingCategory=='MALL':
        return [[[7.0,21.0]]]*7
    elif buildingCategory=='SUPERMRK':
        return [[[7.0,21.0]]]*7
    elif buildingCategory=='PUBLCASM':
        return [[[7.0,21.0]]]*7
    elif buildingCategory=='SCHOOL':
        return [[[7.0,21.0]]]*7
    elif buildingCategory=='LIBRARY':
        return [[[8.0,18.0]]]*7
    elif buildingCategory=='WORSHIP':
        return [[[0.0,21.0]]]*7
    elif buildingCategory=='ACADEMIC':
        return [[[7.0,21.0]]]*7
    elif buildingCategory=='FIREPOL':
        return [[[0.0,24.0]]]*7
    elif buildingCategory=='MUSEUM':
        return [[[7.0,21.0]]]*7
    elif buildingCategory=='ATHLETIC':
        return [[[7.0,21.0]]]*7
    elif buildingCategory=='MEDICAL':
        return [[[7.0,21.0]]]*7
    elif buildingCategory=='GARAGE':
        return [[[7.0, 17.0]]]*7
    elif buildingCategory=='RETAIL':
        return [[[7.0, 21.0]]]*7
    else:
        print 'Unidentified building category'
        print buildingCategory

If using POIs directly without checking manually, uncomment the middle section of code below

In [13]:
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

for tractIndex in range(len(tracts)):
    print tractIndex
    tract = tracts[tractIndex]
    pois = tractPois[tract]
    buildings = tractBuildings[tract]
    # print len(buildings)
    for buildingIndex in range(len(buildings)):
        buildingId = buildings.keys()[buildingIndex]
        tractBuildings[tract][buildingId]['hours'] = {}
        
        #buildingPois = tractBuildings[tract][buildingId]['pois']
        #numPois = len(buildingPois)
        
        ## Determining POI classes associated with building
        #buildingPoiClasses = []
        #for buildingPoi in buildingPois:
        #    tractPois[tract][buildingPoi]
        #    buildingPoiClasses.extend(tractPois[tract][buildingPoi]['types'])
        #buildingPoiClasses = list(set(buildingPoiClasses))
        #buildingPoiClasses = [x for x in buildingPoiClasses if x not in ['establishment', 'point_of_interest']]
        #tractBuildings[tract][buildingId]['poi_types'] = buildingPoiClasses

        openingHours = allocateHours(tractBuildings[tract][buildingId]['use'])
        for dayIndex in range(len(weekdays)):
            weekday = weekdays[dayIndex]
            dayHours = openingHours[dayIndex]
            tractBuildings[tract][buildingId]['hours'][weekday] = dayHours

0


Determining the Per-capita area of each building

In [14]:
areaPerPerson = {}  # Area per person for a Toolbox category
with open('../Data/PCA_BB_SDL_model.csv', 'r') as f:
    f.readline()
    for line in f:
        line = line.strip().split(',')
        line = [x for x in line if x!='']
        areaPerPerson[line[0]] = float(line[1])
            
areaPerPerson.items()

[('SCHOOL', 3.23),
 ('MALL', 6.2),
 ('RESIDENT', 40.0),
 ('OFFICE', 19.23),
 ('GARAGE', 18.6),
 ('ATHLETIC', 4.6),
 ('MEDICAL', 7.8),
 ('HOTEL', 8.06),
 ('RESTRNT', 1.86),
 ('LIBRARY', 9.3),
 ('WORSHIP', 1.4),
 ('ACADEMIC', 19.23),
 ('FIREPOL', 50.0),
 ('SUPERMRK', 12.82),
 ('PUBLCASM', 19.0),
 ('RETAIL', 7.1),
 ('MUSEUM', 13.9)]

Again, if using POIs to calculate PCAs directly, uncomment the commented code below

In [15]:
uses = []
nonResidentialPcas = 0
residentialPcas = 0
for tractIndex in range(len(tracts)):
    print tractIndex
    tract = tracts[tractIndex]
    tractAllocatedPcas = []
    # print len(buildings)
    for buildingIndex in range(len(tractBuildings[tract])):
        buildingId = tractBuildings[tract].keys()[buildingIndex]
        tractBuildings[tract][buildingId]['pca'] = areaPerPerson[ tractBuildings[tract][buildingId]['use'] ]
        if tractBuildings[tract][buildingId]['category']=='res':
            residentialPcas += 1
        else:
            nonResidentialPcas += 1
        tractBuildings[tract][buildingId]['capacity'] = tractBuildings[tract][buildingId]['useArea']/\
                                                        tractBuildings[tract][buildingId]['pca']
print nonResidentialPcas, residentialPcas
            
#         if tractBuildings[tract][buildingId]['category']=='res':
#             tractBuildings[tract][buildingId]['pca'] = RESIDENTIAL_PCA
#             tractBuildings[tract][buildingId]['capacity'] = tractBuildings[tract][buildingId]['useArea']/ \
#                                                                 tractBuildings[tract][buildingId]['pca']
#             tractBuildings[tract][buildingId]['pcaSource'] = 'Residential Use'
#             residentialPcas += 1
#         else:
#             for x in tractBuildings[tract][buildingId]['poi_types']:
#                 buildingToolboxUses.append(poiToToolboxUse[x])
#             areasPerPerson = []
#             for b in buildingToolboxUses:
#                 try:
#                     areasPerPerson.append(areaPerPerson[b])
#                 except:
#                     pass
#             if len(areasPerPerson)>0:
#                 chosenPca = max(areasPerPerson)
#                 tractBuildings[tract][buildingId]['pca'] = chosenPca
#                 if tractBuildings[tract][buildingId]['pca']>0:
#                     tractBuildings[tract][buildingId]['capacity'] = tractBuildings[tract][buildingId]['useArea']/\
#                                                                 tractBuildings[tract][buildingId]['pca']
#                     tractBuildings[tract][buildingId]['pcaSource'] = 'POI'
#                 else:
#                     tractBuildings[tract][buildingId]['capacity'] = \
#                             tractBuildings[tract][buildingId]['useArea']/25
#                     tractBuildings[tract][buildingId]['pcaSource'] = 'POI Zero default'
#                 if chosenPca>0:
#                     tractAllocatedPcas.append(chosenPca)
#                 determinedPcas += 1
    
#     if len(tractAllocatedPcas)>0:
#         random.shuffle(tractAllocatedPcas)
#         for buildingIndex in range(len(tractBuildings[tract])):
#             buildingId = tractBuildings[tract].keys()[buildingIndex]
#             if 'pca' not in tractBuildings[tract][buildingId].keys():
#                 tractBuildings[tract][buildingId]['pca'] = random.choice(tractAllocatedPcas)
#                 tractBuildings[tract][buildingId]['capacity'] = tractBuildings[tract][buildingId]['useArea']/\
#                                                                 tractBuildings[tract][buildingId]['pca']
#                 tractBuildings[tract][buildingId]['pcaSource'] = 'Random tract sample'
#                 randomPcas += 1
#     else:
#         for buildingIndex in range(len(tractBuildings[tract])):
#             buildingId = tractBuildings[tract].keys()[buildingIndex]
#             if 'pca' not in tractBuildings[tract][buildingId].keys():
#                 tractBuildings[tract][buildingId]['pca'] = UNKNOWN_LOCATIONS_PCA
#                 tractBuildings[tract][buildingId]['capacity'] = tractBuildings[tract][buildingId]['useArea']/\
#                                                                 tractBuildings[tract][buildingId]['pca']
#                 tractBuildings[tract][buildingId]['pcaSource'] = 'Constant unknown'
#                 constantPcas += 1

# print determinedPcas, residentialPcas, randomPcas, constantPcas

0
374 287


In [16]:
for tractIndex in range(len(tracts)):
    tract = tracts[tractIndex]
    pickle_file = tract+'.pickle'
    try:
        f = open('../PickledData/HyperlocalBuildings/'+pickle_file, 'wb')
        pickle.dump(tractBuildings[tract], f, pickle.HIGHEST_PROTOCOL)
        f.close()
    except Exception as e:
        print('Unable to save data to', pickle_file, ':', e)
        raise